# Simple iteration for systems of linear equations

First, generate a random diagonally dominant matrix, for testing.

In [1]:
import numpy as np
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

# I.  Jacobi iteration

Given

$$
A x = b
$$

separate the diagonal part $D$,

$$ A = D + (A - D) $$

and write

$$
x = D^{-1} (D - A) x + D^{-1} b\;.
$$

Then iterate

$$
x_{n + 1} = B x_{n} + c\;,
$$

where 

$$
B = D^{-1} (A - D) \qquad \text{and} \qquad c = D^{-1} b
$$


Let's construct the matrix and the r.h.s. for the Jacobi iteration

In [2]:
diag_1d = np.diag(A)

B = -A.copy()
np.fill_diagonal(B, 0)

D = np.diag(diag_1d)
invD = np.diag(1./diag_1d)
BB = invD @ B 
c = invD @ b

In [3]:
# sanity checks
from numpy.testing import assert_allclose

assert_allclose(-B + D, A)


# xx is a "ground truth" solution, compute it using a direct method
xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@xx, b)
np.testing.assert_allclose(D@xx, B@xx + b)
np.testing.assert_allclose(xx, BB@xx + c)

Check that $\| B\| \leqslant 1$:

In [4]:
np.linalg.norm(BB)

0.36436161983015336

### Do the Jacobi iteration

In [5]:
n_iter = 50

x0 = np.ones(n)
x = x0
for _ in range(n_iter):
    x = BB @ x + c

In [6]:
# Check the result:

A @ x - b

array([ 1.11022302e-16,  0.00000000e+00, -2.22044605e-16, -1.11022302e-16,
        1.11022302e-16,  0.00000000e+00, -2.08166817e-17,  0.00000000e+00,
       -2.77555756e-17,  1.11022302e-16])

### Task I.1

Collect the proof-of-concept above into a single function implementing the Jacobi iteration. This function should receive the r.h.s. matrix $A$, the l.h.s. vector `b`, and the number of iterations to perform.


The matrix $A$ in the illustration above is strongly diagonally dominant, by construction. 
What happens if the diagonal matrix elements of $A$ are made smaller? Check the convergence of the Jacobi iteration, and check the value of the norm of $B$.

(20% of the total grade)


In [7]:
# ... ENTER YOUR CODE HERE ...
def Jacobi_iteration(A, b, num_iter=50, verbose=False):
    B = -A.copy()
    np.fill_diagonal(B, 0)
    
    diag = np.diag(A)
    inv_diag = np.diag(1./diag)
    B = np.dot(inv_diag, B)
    c = np.dot(inv_diag, b)
    
    if verbose:
        print("norm B:", np.linalg.norm(B, ord=np.inf))
        
    x_new = np.ones_like(b)
    for i in range(num_iter):
        x_prev = x_new.copy()
        x_new = np.dot(B, x_prev) + c
        if verbose:
            print("%d:" % i, np.linalg.norm(x_new - x_prev))
    return x_new

In [8]:
x = Jacobi_iteration(A, b)
xx = np.linalg.solve(A, b)

np.allclose(x, xx)

True

Let's check convergense of Jacobi method:

In [9]:
x = Jacobi_iteration(A, b, verbose=True)

norm B: 0.3969558732098327
0: 4.017670932150149
1: 1.2439262407070686
2: 0.37150589709834797
3: 0.11149610276038871
4: 0.033437377185133677
5: 0.01002777263094849
6: 0.0030074056173186965
7: 0.0009019337306094896
8: 0.0002704944221843025
9: 8.112259066743358e-05
10: 2.432906062204276e-05
11: 7.2964038372576306e-06
12: 2.1882270753681245e-06
13: 6.562599651808278e-07
14: 1.9681556216645286e-07
15: 5.902594636795646e-08
16: 1.770216900721955e-08
17: 5.3089667635383426e-09
18: 1.5921850085505798e-09
19: 4.775040495842271e-10
20: 1.4320579748219748e-10
21: 4.294811884741762e-11
22: 1.2880351114952751e-11
23: 3.86287896476393e-12
24: 1.1584939050400663e-12
25: 3.474423252972769e-13
26: 1.0420575239324089e-13
27: 3.125362897997318e-14
28: 9.370374086345794e-15
29: 2.80638747251276e-15
30: 8.42665475904218e-16
31: 2.515543441319542e-16
32: 7.238184373059934e-17
33: 1.9778890766513102e-17
34: 8.673617379884035e-18
35: 3.469446951953614e-18
36: 0.0
37: 0.0
38: 0.0
39: 0.0
40: 0.0
41: 0.0
42: 0.

Let's look what happens if matrix A is not diagonally dominant:

In [10]:
A1 = rndm.uniform(size=(n, n))
x1 = Jacobi_iteration(A1, b, verbose=True)
xx1 = np.linalg.solve(A1, b)

norm B: 16.78389594884626
0: 30.614999981715687
1: 260.14453080221784
2: 2241.7051872050074
3: 19271.37004107396
4: 165572.18421145683
5: 1423137.192151448
6: 12230378.049585639
7: 105111360.60152042
8: 903350809.7034506
9: 7763606477.923543
10: 66722240220.17173
11: 573426428152.5233
12: 4928159939361.045
13: 42353751442368.37
14: 363997979201504.2
15: 3128283196103139.5
16: 2.688519253610676e+16
17: 2.3105759051097872e+17
18: 1.9857626111744604e+18
19: 1.7066105204424956e+19
20: 1.4667007285237663e+20
21: 1.260516679865771e+21
22: 1.0833173184682614e+22
23: 9.310280706624536e+22
24: 8.001471531785953e+23
25: 6.876650521227191e+24
26: 5.9099532133859e+25
27: 5.079151089123144e+26
28: 4.365140442687345e+27
29: 3.751503105546363e+28
30: 3.224128922243716e+29
31: 2.7708912973789246e+30
32: 2.381368353144869e+31
33: 2.0466032856374438e+32
34: 1.7588984095007708e+33
35: 1.5116381551106307e+34
36: 1.299136948241848e+35
37: 1.1165084743204446e+36
38: 9.595533210847459e+36
39: 8.2466241607809

In [11]:
error = xx1 - x1
error

array([-7.71472847e+46, -2.19650221e+46, -2.70898688e+46, -3.44764908e+46,
       -4.31425974e+46, -4.17765861e+46, -7.64567150e+46, -3.15125314e+46,
       -3.31634010e+46, -8.04173396e+46])

As can be seen, difference between iterations grows and it does not converge to the true answer, since the norm of B is far bigger than 1

# II. Seidel's iteration.

##### Task II.1

Implement the Seidel's iteration. 

Test it on a random matrix. Study the convergence of iterations, relate to the norm of the iteration matrix.

(30% of the total grade)

In [12]:
# ... ENTER YOUR CODE HERE ...
def Seidel_iteration(A, b, num_iter=50, verbose=False, verbose_B=False):
    B = -A.copy()
    np.fill_diagonal(B, 0)
    B = np.dot(np.diag(1./np.diag(A)), B)
    
    if verbose_B:
        print("norm B:", np.linalg.norm(B, ord=np.inf))
    
    x_new = np.ones_like(b)
    for i in range(num_iter):
        x_prev = x_new.copy()
        for j in range(len(b)):
            x_new[j] = (b[j] - np.dot(A[j, j+1:], x_prev[j+1:]) - np.dot(A[j, :j], x_prev[:j]))/A[j, j]
            
        if verbose:
            print(np.linalg.norm(x_new - x_prev))
        
    return x_new

In [13]:
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

In [14]:
x = Seidel_iteration(A, b, verbose_B=True)
xx = np.linalg.solve(A, b)

norm B: 0.3742909530999712


In [15]:
np.allclose(x, xx)

True

In [16]:
np.allclose(np.dot(A, x), b)

True

# III. Minimum residual scheme

### Task III.1

Implement the $\textit{minimum residual}$ scheme: an explicit non-stationary method, where at each step you select the iteration parameter $\tau_n$ to minimize the residual $\mathbf{r}_{n+1}$ given $\mathbf{r}_n$. Test it on a random matrix, study the convergence to the solution, in terms of the norm of the residual and the deviation from the ground truth solution (which you can obtain using a direct method). Study how the iteration parameter $\tau_n$ changes as iterations progress.

(50% of the grade)

In [17]:
# ... ENTER YOUR CODE HERE ...
def min_res_iteration(A, b, ground_trurh=None, eps=1e-8, num_iter=50, verbose=False, early_break=False):
    x_new = np.ones_like(b)
    
    for i in range(num_iter):
        x_prev = x_new.copy()
        
        residual = np.dot(A, x_prev) - b
        A_res = np.dot(A, residual)
        tau = np.dot(residual, A_res)/np.dot(A_res, A_res)
        
        if verbose:
            print("%d:" %i, "residual norm: %f" % np.linalg.norm(residual), "tau: %f" % tau)
            if ground_trurh is not None:
                print("deviation: %.8f" % np.linalg.norm(ground_trurh - x_prev))
            print("----------------------------------")
        
        x_new = x_prev - tau*residual
        
        if early_break:
            if np.linalg.norm(x_new - x_prev) < eps:
                print("early break at %d iteration" % i)
                break
        
    return x_new

In [18]:
A2 = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b2 = rndm.uniform(size=n)

In [19]:
x2 = min_res_iteration(A2, b2)
xx2 = np.linalg.solve(A2, b2)

np.allclose(x2, xx2, atol=1e-8)

True

In [22]:
min_res_iteration(A2, b2, ground_trurh=xx2, verbose=True, early_break=False)

0: residual norm: 62.526919 tau: 0.048929
deviation: 3.05841903
----------------------------------
1: residual norm: 1.530105 tau: 0.066696
deviation: 0.10320236
----------------------------------
2: residual norm: 0.137148 tau: 0.054084
deviation: 0.00781558
----------------------------------
3: residual norm: 0.018842 tau: 0.059218
deviation: 0.00118823
----------------------------------
4: residual norm: 0.003160 tau: 0.056548
deviation: 0.00019174
----------------------------------
5: residual norm: 0.000513 tau: 0.058791
deviation: 0.00003265
----------------------------------
6: residual norm: 0.000096 tau: 0.057094
deviation: 0.00000590
----------------------------------
7: residual norm: 0.000016 tau: 0.058203
deviation: 0.00000099
----------------------------------
8: residual norm: 0.000003 tau: 0.057532
deviation: 0.00000018
----------------------------------
9: residual norm: 0.000000 tau: 0.057636
deviation: 0.00000003
----------------------------------
10: residual norm: 

array([ 0.02331187,  0.04463403,  0.0494192 ,  0.03055605,  0.0485216 ,
        0.03657139, -0.00903743,  0.04390046,  0.01372572,  0.04851064])

As can be seen, norm of the residual converges to zero as iteration progress. As for tau, it grows firstly, then remains almost still, since iterations get quite close to the true solution.